# ODTC Tutorial: Connect, Lid, Block Temperature, and Protocol

This notebook walks through the ODTC (Inheco) thermocycler interface: setup, listing methods, lid (door) commands, setting block temperature, and running a protocol. Use it as a reference for the recommended workflow.

## 0. Logging (optional)

**Optional.** Run this cell if you want to see backend/enriched progress and instrument (SiLA) communication in a file. Backend and SiLA log to one timestamped file; progress (ODTCProgress) is logged every 150 s while you await a handle. Optional: set **`tc.backend.data_event_log_path`** (e.g. in section 1) for raw DataEvent JSONL.

In [1]:
import logging
from datetime import datetime

# One log file and one DataEvents file per notebook run (timestamped)
_run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
odtc_log_file = f"odtc_run_{_run_id}.log"
odtc_data_events_file = f"odtc_data_events_{_run_id}.jsonl"

# File handler: ODTC backend + thermocycling + SiLA (storage inheco used during setup)
_fh = logging.FileHandler(odtc_log_file, encoding="utf-8")
_fh.setLevel(logging.DEBUG)  # Set to logging.INFO to reduce verbosity
_fh.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s"))

for _name in ("pylabrobot.thermocycling.inheco", "pylabrobot.storage.inheco"):
    _log = logging.getLogger(_name)
    _log.setLevel(logging.DEBUG)
    _log.addHandler(_fh)

print(f"Logging to {odtc_log_file}")
print(f"DataEvent summaries go to the same log; set tc.backend.data_event_log_path = '{odtc_data_events_file}' for raw JSONL (optional)")

Logging to odtc_run_20260215_004101.log
DataEvent summaries go to the same log; set tc.backend.data_event_log_path = 'odtc_data_events_20260215_004101.jsonl' for raw JSONL (optional)


## 1. Imports and thermocycler

Use **ODTCThermocycler** (recommended): it owns `odtc_ip`, `variant`, and dimensions. Alternative: generic `Thermocycler` + `ODTCBackend` for custom backend options.

In [2]:
import logging
from pylabrobot.resources import Coordinate
from pylabrobot.thermocycling.inheco import ODTCThermocycler
# Preferred: ODTCThermocycler (built-in ODTC dimensions; variant 96 or 384)
tc = ODTCThermocycler(name="odtc_test", odtc_ip="192.168.1.50", variant=96, child_location=Coordinate.zero())
# Optional: raw DataEvent JSONL (path from section 0)
tc.backend.data_event_log_path = odtc_data_events_file
# Override: tc = Thermocycler(..., backend=ODTCBackend(odtc_ip=..., variant=96, logger=...), ...) for custom backend

## 2. Connect and list device methods and premethods

`setup()` resets and initializes the device. **Methods** = runnable protocols on the device; **premethods** = setup-only (e.g. set temperature). Use **`tc.backend.list_protocols()`** to get a **`ProtocolList`** (`.methods`, `.premethods`, `.all`); **`tc.backend.get_protocol(name)`** returns an **ODTCProtocol** for methods (`None` for premethods).

In [3]:
await tc.setup()
print("✓ Connected and initialized.")

2026-02-15 00:41:02,166 - pylabrobot.storage.inheco.scila.inheco_sila_interface - INFO - Device reset (unlocked)
2026-02-15 00:41:02,188 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - GetStatus returned raw state: 'standby' (type: str)
2026-02-15 00:41:02,188 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device is in standby state, calling Initialize...
2026-02-15 00:41:02,580 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - Device successfully initialized and is in idle state


✓ Connected and initialized.


**ODTCProtocol and running options:** **`tc.backend.get_protocol(name)`** returns an **ODTCProtocol** (subclasses `Protocol`; has `.steps`, `.name`, and ODTC-specific fields). Use **`print(odtc)`** for a human-readable summary.

- **Roundtrip:** **`tc.run_protocol(odtc, block_max_volume)`** — pass the ODTCProtocol from the device; same device-calculated config (thermal tuning preserved).
- **Run by name (recommended for PCR):** **`tc.run_stored_protocol("MethodName")`** — device runs its stored method; optimal thermal (overshoots and device-tuned ramps).
- **Custom protocol:** **`tc.run_protocol(protocol, block_max_volume, config=...)`** with a generic `Protocol` and optional **`tc.backend.get_default_config(post_heating=...)`** — no prior device config means default overshoot; use roundtrip or run-by-name for best thermal performance.

In [4]:
protocol_list = await tc.backend.list_protocols()
# Print methods and premethods in clear sections
print(protocol_list)

# Iteration and .all still work: for name in protocol_list, protocol_list.all
# Optional: inspect a runnable method (get_protocol returns None for premethods)
# ODTCProtocol subclasses Protocol; print(odtc) shows full structure and steps
if protocol_list.all:
    first_name = protocol_list.methods[0] if protocol_list.methods else protocol_list.premethods[0]
    fetched_protocol = await tc.backend.get_protocol(first_name)
    if fetched_protocol is not None:
        print("\nExample stored protocol (full structure and steps):")
        print(fetched_protocol)
        # Roundtrip: run with same ODTC config via run_protocol(odtc, block_max_volume)


Methods (runnable protocols):
  - plr_currentProtocol
  - M18_Abnahmetest
  - M23_LEAK
  - M24_LEAKCYCLE
  - M22_A-RUNIN
  - M22_B-RUNIN
  - M30_PCULOAD
  - M33_Abnahmetest
  - M34_Dauertest
  - M35_VCLE
  - M36_Verifikation
  - M37_BGI-Kon
  - M39_RMATEST
  - M18_Abnahmetest_384
  - M23_LEAK_384
  - M22_A-RUNIN_384
  - M22_B-RUNIN_384
  - M30_PCULOAD_384
  - M33_Abnahmetest_384
  - M34_Dauertest_384
  - M35_VCLE_384
  - M36_Verifikation_384
  - M37_BGI-Kon_384
  - M39_RMATEST_384
  - M120_OVT
  - M320_OVT
  - M121_OVT
  - M321_OVT
  - M123_OVT
  - M323_OVT
  - M124_OVT
  - M324_OVT
  - M125_OVT
  - M325_OVT
  - PMA cycle
  - Test
  - DC4_ProK_digestion
  - DC4_3Prime_Ligation
  - DC4_ProK_digestion_1_test
  - DC4_3Prime_Ligation_test
  - DC4_5Prime_Ligation_test
  - DC4_USER_Ligation_test
  - DC4_5Prime_Ligation
  - DC4_USER_Ligation
  - DC4_ProK_digestion_37
  - DC4_ProK_digestion_60
  - DC4_3Prime_Ligation_Open_Close
  - DC4_3Prime_Ligation_37
  - DC4_5Prime_Ligation_37
  - DC4_USER

## 3. Lid (door) commands

The Thermocycler API uses **`open_lid`** / **`close_lid`** (ODTC device calls this the door). Door open/close use a 60 s estimated duration and corresponding timeout. Use **`wait=False`** to get an execution handle and avoid blocking; then **`await handle.wait()`** or **`await handle`** when you need to wait. For method runs, progress is logged every **progress_log_interval** (default 150 s) while you await. Omit `wait=False` to block until the command finishes.

**Waiting:** `await handle.wait()` or `await handle` (same). For method/protocol runs, **`await tc.wait_for_profile_completion(poll_interval=..., timeout=...)`** uses polling and supports a timeout.

In [5]:
# Non-blocking: returns execution handle (ODTCExecution); await handle.wait() when you need to wait
door_handle = await tc.close_lid(wait=False)
print(f"Close started (request_id={door_handle.request_id})")

Close started (request_id=329628599)


## 4. Block temperature and protocol

ODTC has no direct “set block temp” command; **`set_block_temperature`** uploads and runs a PreMethod. Use **`wait=False`** to get a handle; **`run_protocol(protocol, block_max_volume, config=...)`** is always non-blocking — await the returned handle or **`tc.wait_for_profile_completion()`** to block. When you **await** a method or premethod handle, progress is reported every **progress_log_interval** (default 150 s); for premethods the **target** shown is the premethod's target (e.g. 37°C), not the device's ramp setpoint. Use **`config=tc.backend.get_default_config(post_heating=True)`** to hold temperatures after the method ends.

In [6]:
await door_handle.wait()
print("Door closed.")

# set_block_temperature runs a premethod; wait=False returns execution handle (ODTCExecution)
# Override: debug_xml=True, xml_output_path="out.xml" to save generated MethodSet XML
mount_handle = await tc.set_block_temperature([37.0],
                                              wait=False,
                                              debug_xml=True,
                                              xml_output_path="debug_set_mount_temp.xml"
                                              )
block = await tc.get_block_current_temperature()
lid = await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")

2026-02-15 00:41:10,960 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - [2026-02-15 00:41:10] Waiting for command
  Command: CloseDoor
  Duration (timeout): 120.0s
  Remaining: 120s
2026-02-15 00:41:18,952 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - MethodSet XML saved to: debug_set_mount_temp.xml


Door closed.
Block: 28.1 °C  Lid: 60.9 °C


In [7]:
from pylabrobot.thermocycling.standard import Protocol, Stage, Step

# Wait for set_block_temperature (previous cell) to finish before starting a protocol
await mount_handle

# run_protocol is always non-blocking; returns execution handle (ODTCExecution). To block: await handle.wait() or tc.wait_for_profile_completion()
config = tc.backend.get_default_config(post_heating=False)  # if True: hold temps after method ends
cycle_protocol = Protocol(stages=[
    Stage(steps=[
        Step(temperature=[37.0], hold_seconds=10.0),
        Step(temperature=[60.0], hold_seconds=10.0),
        Step(temperature=[10.0], hold_seconds=10.0),
    ], repeats=1)
])
execution = await tc.run_protocol(cycle_protocol, 50.0, config=config)
# Override: run_stored_protocol("MethodName") to run a device-stored method by name
print(f"Protocol started (request_id={execution.request_id})")
block, lid = await tc.get_block_current_temperature(), await tc.get_lid_current_temperature()
print(f"Block: {block[0]:.1f} °C  Lid: {lid[0]:.1f} °C")


2026-02-15 00:41:41,819 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - [2026-02-15 00:41:41] Waiting for command
  Command: plr_currentProtocol (ExecuteMethod)
  Duration (timeout): 655.374s
  Remaining: 648s
2026-02-15 00:41:41,820 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - ODTC progress: elapsed 5s, step 1/1, cycle 1/1, setpoint 37.0°C, block 27.6°C, lid 59.7°C
2026-02-15 00:44:11,825 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - ODTC progress: elapsed 152s, step 1/1, cycle 1/1, setpoint 37.0°C, block 37.0°C, lid 109.5°C
2026-02-15 00:46:41,831 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - ODTC progress: elapsed 305s, step 1/1, cycle 1/1, setpoint 37.0°C, block 37.0°C, lid 109.6°C
2026-02-15 00:49:11,836 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - ODTC progress: elapsed 452s, step 1/1, cycle 1/1, setpoint 37.0°C, block 37.0°C, lid 109.9°C


Protocol started (request_id=462796715)
Block: 37.0 °C  Lid: 110.2 °C


**Status and progress during a protocol run** — When you **await execution** (or **await execution.wait()**), progress is logged every **progress_log_interval** (default 150 s) from the latest DataEvent. **`get_progress_snapshot()`** is the single readout for progress during a run: it returns **ODTCProgress** (elapsed_s, current_temp_c, target_temp_c, lid_temp_c; step/cycle/hold when protocol is registered). Poll with **`is_profile_running()`** and **`get_progress_snapshot()`**. For a direct sensor read outside a run, use **`get_block_current_temperature()`** / **`get_lid_current_temperature()`**. Run the cell below after starting the protocol to poll manually.

## 4b. Logging (reference)

Progress and DataEvents use the same log file (section 0). For raw DataEvent JSONL, set **`tc.backend.data_event_log_path`** before a run (e.g. in section 1).

In [8]:
import asyncio

# Poll status a few times while the protocol runs (run this cell after starting the protocol)
# get_progress_snapshot() returns ODTCProgress; print(snap) uses __str__ (same as progress logged every 150 s)
for poll in range(10):
    running = await tc.is_profile_running()
    if not running:
        print(f"Poll {poll + 1}: profile no longer running.")
        break
    snap = await tc.get_progress_snapshot()
    if snap:
        print(f"Poll {poll + 1}: {snap}")
    else:
        print(f"Poll {poll + 1}: no progress snapshot yet.")
    await asyncio.sleep(5)
else:
    print("Still running after 6 polls; use await execution.wait() or tc.wait_for_profile_completion() to block until done.")

Poll 1: ODTC progress: elapsed 4s, step 1/3, cycle 1/1, setpoint 37.0°C, block 37.0°C, lid 110.2°C
Poll 2: ODTC progress: elapsed 9s, step 1/3, cycle 1/1, setpoint 37.0°C, block 37.0°C, lid 110.2°C
Poll 3: ODTC progress: elapsed 14s, step 2/3, cycle 1/1, setpoint 55.0°C, block 54.9°C, lid 110.1°C
Poll 4: ODTC progress: elapsed 19s, step 2/3, cycle 1/1, setpoint 60.0°C, block 60.3°C, lid 110.1°C
Poll 5: ODTC progress: elapsed 25s, step 2/3, cycle 1/1, setpoint 60.0°C, block 60.1°C, lid 110.1°C
Poll 6: ODTC progress: elapsed 30s, step 3/3, cycle 1/1, setpoint 50.5°C, block 51.6°C, lid 110.1°C
Poll 7: ODTC progress: elapsed 35s, step 3/3, cycle 1/1, setpoint 39.5°C, block 40.8°C, lid 110.0°C
Poll 8: ODTC progress: elapsed 40s, step 3/3, cycle 1/1, setpoint 28.3°C, block 31.9°C, lid 109.9°C
Poll 9: ODTC progress: elapsed 45s, step 3/3, cycle 1/1, setpoint 16.9°C, block 24.4°C, lid 109.8°C
Poll 10: ODTC progress: elapsed 50s, step 3/3, cycle 1/1, setpoint 10.0°C, block 18.6°C, lid 109.7°C
S

## 5. Wait, open lid, disconnect

Await protocol completion, open the lid (non-blocking then wait), then **`tc.stop()`** to close the connection.

In [9]:
# Block until protocol done; progress is logged every 150 s (progress_log_interval) while waiting
# Alternatively: await tc.wait_for_profile_completion(poll_interval=..., timeout=...)
await execution.wait()

open_handle = await tc.open_lid(wait=False)
await open_handle.wait()
await tc.stop()
print("Done.")

2026-02-15 00:50:34,070 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - [2026-02-15 00:50:34] Waiting for command
  Command: plr_currentProtocol (ExecuteMethod)
  Duration (timeout): 113.69154545454546s
  Remaining: 56s
2026-02-15 00:50:34,072 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - ODTC progress: elapsed 55s, step 3/3, cycle 1/1, setpoint 10.0°C, block 13.9°C, lid 109.8°C
2026-02-15 00:50:36,788 - pylabrobot.thermocycling.inheco.odtc_backend - INFO - [2026-02-15 00:50:36] Waiting for command
  Command: OpenDoor
  Duration (timeout): 120.0s
  Remaining: 120s


Done.
